# Input and output formats

This section provides an overview of the input and output formats
supported by DFTK, usually via integration with a third-party library.

## Reading input formats supported by ASE
ASE is short for the
[atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html),
a Python package to simplify the process of setting up, running and
analysing results from atomistic simulations across different programs.
If it is installed it is automatically used by DFTK in order to read a wide range
of input files, such as xyz, CIF, input files of various other codes
(e.g. Quantum Espresso, VASP, ABINIT, CASTEP, ...).
The full list of formats
can be found in the [ASE IO documentation](https://wiki.fysik.dtu.dk/ase/ase/io/io.html).

As an example we start the calculation of a simple antiferromagnetic iron crystal
using a Quantum-Espresso input file, [Fe_afm.pwi](Fe_afm.pwi).
From this file the lattice, atomic positions and the initial magnetisation are read.
For more details about calculations on magnetic systems
using collinear spin, see Collinear spin and magnetic systems.

First we parse the Quantum Espresso input file to DFTK datastructures:

In [1]:
using DFTK

qe_input  = "Fe_afm.pwi"
atoms     = load_atoms(qe_input)
positions = load_positions(qe_input)
lattice   = load_lattice(qe_input)
magnetic_moments = load_magnetic_moments(qe_input)

2-element Vector{StaticArrays.SVector{3, Float64}}:
 [0.0, 0.0, -9.6]
 [0.0, 0.0, -9.6]

At this point a file of any format supported by ASE could be passed instead,
e.g. an `xyz` file or an ABINIT input file. Behind the scenes ASE takes care
to select the right parser and extract the required structural information.

Next we attach the pseudopotential information, since this information is currently
not exposed inside the ASE datastructures.
See Creating slabs with ASE for more details.

In [2]:
atoms = map(atoms) do el
    @assert el.symbol == :Fe
    ElementPsp(:Fe, psp=load_psp("hgh/pbe/fe-q16.hgh"))
end;

Finally we run the calculation.

In [3]:
model = model_LDA(lattice, atoms, positions; magnetic_moments, temperature=0.01)
basis = PlaneWaveBasis(model; Ecut=10, kgrid=(2, 2, 2))
ρ0 = guess_density(basis, magnetic_moments)
scfres = self_consistent_field(basis, tol=1e-4, ρ=ρ0, mixing=KerkerMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag
---   ---------------   ---------   ---------   ------   ----
  1   -223.7481251797                    0.22   -6.320    4.8
  2   -224.1772718812       -0.37       -0.23   -3.269    1.7
  3   -224.2162787092       -1.41       -1.07   -1.801    2.7
  4   -224.2192866470       -2.52       -1.36   -1.353    2.2
  5   -224.2205739132       -2.89       -1.67   -0.898    1.3
  6   -224.2199931366   +   -3.24       -1.71   -0.636    1.0
  7   -224.2197562215   +   -3.63       -1.70   -0.554    1.6
  8   -224.2212925914       -2.81       -2.13   -0.322    1.0
  9   -224.2214136286       -3.92       -2.63   -0.048    1.1
 10   -224.2214166997       -5.51       -2.96   -0.009    1.1


!!! note "DFTK and ASE"
    DFTK also supports using ASE to setup a calculation
    and provides two-way conversion routines between key DFTK
    and ASE datastructures. See Creating slabs with ASE for details.

## Writing VTK files for visualization
For visualizing the density or the Kohn-Sham orbitals DFTK supports storing
the result of an SCF calculations in the form of VTK files.
These can afterwards be visualized using tools such
as [paraview](https://www.paraview.org/).
Using this feature requires
the [WriteVTK.jl](https://github.com/jipolanco/WriteVTK.jl/) Julia package.

In [4]:
using WriteVTK
save_scfres("iron_afm.vts", scfres; save_ψ=true);

This will save the iron calculation above into the file `iron_afm.vts`,
using `save_ψ=true` to also include the KS orbitals.

## Writing and reading JLD2 files
The full state of a DFTK self-consistent field calculation can be
stored on disk in form of an [JLD2.jl](https://github.com/JuliaIO/JLD2.jl) file.
This file can be read from other Julia scripts
as well as other external codes supporting the HDF5 file format
(since the JLD2 format is based on HDF5).

In [5]:
using JLD2
save_scfres("iron_afm.jld2", scfres);

Since such JLD2 can also be read by DFTK to start or continue a calculation,
these can also be used for checkpointing or for transferring results
to a different computer.
See Saving SCF results on disk and SCF checkpoints for details.

(Cleanup files generated by this notebook)

In [6]:
rm("iron_afm.vts")
rm("iron_afm.jld2")